# LangGraph + MCP Agent System Examples

This notebook demonstrates how to use the minimal agent system that combines LangGraph state management with MCP-style tool interfaces.

In [ ]:
# Import the agent system
from agent_system import Agent, Tool, AgentSystem, create_python_tool, create_memory_tool
import asyncio
import json

## 1. Creating a Simple Agent with Tools

In [ ]:
# Create a simple calculator tool
def calculator_tool(expression: str) -> str:
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Wrap it as a Tool
calc_tool = Tool(
    name="calculator",
    func=calculator_tool,
    description="Evaluates mathematical expressions"
)

# Create an agent with context and tools
math_agent = Agent(
    name="MathExpert",
    context={"role": "mathematician", "expertise": ["arithmetic", "algebra"]},
    tools=[calc_tool]
)

# Run a task
result = await math_agent.run("Please use the calculator to compute 25 * 4 + 10")
print(json.dumps(result, indent=2))

## 2. Creating Custom Tools

In [ ]:
# Create a weather tool (mock)
def weather_tool(location: str) -> str:
    # In reality, this would call a weather API
    weather_data = {
        "new york": "Sunny, 72°F",
        "london": "Cloudy, 59°F",
        "tokyo": "Rainy, 65°F"
    }
    location_lower = location.lower()
    return weather_data.get(location_lower, f"No weather data for {location}")

# Create a string manipulation tool
def string_tool(action: str, text: str) -> str:
    if action == "reverse":
        return text[::-1]
    elif action == "uppercase":
        return text.upper()
    elif action == "wordcount":
        return f"Word count: {len(text.split())}"
    else:
        return "Unknown action"

# Create tools
weather = Tool(name="weather", func=weather_tool, description="Get weather information")
string_ops = Tool(name="string", func=string_tool, description="String operations")

# Create an agent with multiple tools
assistant = Agent(
    name="GeneralAssistant",
    context={"role": "helpful assistant"},
    tools=[weather, string_ops, calc_tool]
)

# Test multiple tools
result = await assistant.run("What's the weather in London?")
print("Weather query:", result["response"])

## 3. Agent as a Tool (Hierarchical Agents)

In [ ]:
# Create specialized agents
research_agent = Agent(
    name="Researcher",
    context={"role": "research specialist", "skills": ["web search", "summarization"]},
    tools=[string_ops]  # Give it some basic tools
)

coding_agent = Agent(
    name="Coder",
    context={"role": "software developer", "languages": ["python", "javascript"]},
    tools=[create_python_tool()]  # Give it code execution
)

# Convert agents to tools
research_tool = research_agent.as_tool()
coding_tool = coding_agent.as_tool()

# Create a manager agent that can delegate to others
manager = Agent(
    name="ProjectManager",
    context={"role": "project coordinator"},
    tools=[research_tool, coding_tool, calc_tool]
)

# Manager delegates tasks
result = await manager.run("Use agent_Researcher to analyze the word count of 'Hello World'")
print("Manager delegation result:")
print(json.dumps(result, indent=2))

## 4. Using the AgentSystem for Coordination

In [ ]:
# Create an agent system
system = AgentSystem()

# Register a shared tool that all agents can use
memory_tool = create_memory_tool()
system.register_shared_tool(memory_tool)

# Create agents through the system
analyst = system.create_agent(
    name="DataAnalyst",
    context={"role": "data analysis"},
    tools=[calc_tool]
)

writer = system.create_agent(
    name="Writer",
    context={"role": "content creation"},
    tools=[string_ops]
)

# Both agents have access to the shared memory tool
# Analyst stores a result
result1 = await analyst.run("Use memory to store key='analysis_result' value='42 data points processed'")
print("Analyst stored:", result1["response"])

# Writer retrieves it
result2 = await writer.run("Use memory to retrieve key='analysis_result'")
print("Writer retrieved:", result2["response"])

## 5. Creating a Hierarchical Team with the System

In [ ]:
# Create a hierarchical team
supervisor = system.create_hierarchical_agents(
    supervisor_name="TeamLead",
    worker_names=["Developer", "Tester", "Designer"],
    supervisor_context={"role": "technical lead", "team_size": 3},
    worker_contexts={
        "Developer": {"skills": ["python", "javascript"]},
        "Tester": {"skills": ["unit testing", "integration testing"]},
        "Designer": {"skills": ["UI/UX", "prototyping"]}
    }
)

# Supervisor can now delegate to team members
result = await supervisor.run("Use agent_Developer to create a hello world function")
print("Team collaboration result:")
print(json.dumps(result, indent=2))

## 6. Creating Complex Tool Chains

In [ ]:
# Create a tool that uses other agents
async def research_and_code_tool(topic: str) -> str:
    # First, research the topic
    research_result = await research_agent.run(f"Research about {topic}")
    
    # Then, generate code based on research
    code_prompt = f"Based on this research: {research_result['response']}, write Python code for {topic}"
    code_result = await coding_agent.run(code_prompt)
    
    return f"Research: {research_result['response']}\nCode: {code_result['response']}"

# Create a composite tool
composite_tool = Tool(
    name="research_and_code",
    func=research_and_code_tool,
    description="Research a topic and generate code"
)

# Create an agent with the composite tool
advanced_agent = Agent(
    name="AdvancedAssistant",
    context={"capabilities": "research and development"},
    tools=[composite_tool]
)

# Use the composite tool
result = await advanced_agent.run("Use research_and_code for 'sorting algorithms'")
print("Composite tool result:")
print(result["response"])

## Key Concepts Demonstrated

1. **Agent Creation**: Agents have context (their knowledge/role) and tools (their capabilities)
2. **Tool Creation**: Tools are simple functions wrapped with metadata
3. **Agents as Tools**: Any agent can become a tool for another agent, enabling delegation
4. **Shared Tools**: The AgentSystem allows tools to be shared across agents
5. **Hierarchical Teams**: Supervisors can coordinate worker agents
6. **Composite Tools**: Tools can orchestrate multiple agents for complex tasks

This minimal implementation shows how LangGraph's state management can be combined with MCP-style tool interfaces to create flexible agent systems.